In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

### Đọc dữ liệu

In [4]:
file_path = './ProcessedData.csv'
data = pd.read_csv(file_path)

### Xử lý dữ liệu

In [5]:
# Remove unnecessary columns and prepare features and target
# Dropping 'Unnamed: 0', 'timestamp', and 'Weather Description' for now
columns_to_drop = ['Unnamed: 0', 'timestamp', 'Weather Description']
air = ['so2','no2','pm10','pm2_5','o3','co']
weather = ['Temprature (Kelvin)','Feels like (Kelvin)','Pressure (mb)',
           'Humidity (%)','Temperature Amplitude','Wind speed (m/s)','Wind degree',
           'Wind gust (m/s)','Clouds all (%)','Rain 1h (mm)']
features = data.drop(columns=columns_to_drop + ['aqi'] + air + ['no'] + ['nh3'])  # Exclude target column 'aqi' and air
target = data[air]

### Xây dựng hàm loss

#### Dictionary để tham chiếu AQI dựa trên các chỉ số thành phần không khí

In [6]:
aqi_thresholds = {
    'so2': [
        (0, 20), (20, 80), (80, 250), (250, 350), (350, float('inf'))
    ],
    'no2': [
        (0, 40), (40, 70), (70, 150), (150, 200), (200, float('inf'))
    ],
    'pm10': [
        (0, 20), (20, 50), (50, 100), (100, 200), (200, float('inf'))
    ],
    'pm2_5': [
        (0, 10), (10, 25), (25, 50), (50, 75), (75, float('inf'))
    ],
    'o3': [
        (0, 60), (60, 100), (100, 140), (140, 180), (180, float('inf'))
    ],
    'co': [
        (0, 4400), (4400, 9400), (9400, 12400), (12400, 15400), (15400, float('inf'))
    ]
}

#### Trả về danh sách các chất gây ra ô nhiễm dựa trên các thành phần không khí trong 1 dòng

In [7]:
def identify_pollutants(row, thresholds, pollutant_names):
    """
    Xác định mức AQI của một mảng dữ liệu và trả về các chất ô nhiễm với mức AQI tương ứng.

    Parameters:
    - row: list hoặc numpy array chứa các giá trị của chất ô nhiễm.
    - thresholds: từ điển ngưỡng AQI.
    - pollutant_names: danh sách tên các chất ô nhiễm tương ứng với row.

    Returns:
    - Danh sách các chất ô nhiễm cùng với mức AQI của chúng.
    """
    aqi_details = []  # Danh sách lưu trữ chi tiết các chất ô nhiễm và mức AQI
    for i, pollutant in enumerate(pollutant_names):
        value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i
        for index, (lower, upper) in enumerate(thresholds[pollutant], start=1):
            if lower <= value < upper:
                if index >= 4:
                    aqi_details.append(pollutant)  # Lưu tên chất ô nhiễm và mức AQI
                break
    return aqi_details  # Trả về danh sách các chất ô nhiễm và mức AQI

#### Xác định ô nhiễm gây ra bởi các nhóm chất nào (bụi mịn (PM2.5, PM10)/ Khí độc (CO, SO2, NO)/ Ozone tầng mặt đất (O3))
Các chất ô nhiễm cùng nhóm có các nguyên nhân gây ra và biện pháp phòng tránh gần tương tự nhau

In [8]:
def group_pollutant(pollutants):
    groups = [['pm2_5', 'pm10'],['co','so2','no2'], ['o3']]
    idx_array = []
    for item in pollutants:
        for index in range(len(groups)):
            if item in groups[index] and index not in idx_array:
                idx_array.append(index)

    return idx_array

#### Hàm loss
Trả ra kết quả đúng/sai. Nếu mẫu không khí dự đoán và mẫu không khí thực tế có cùng các nhóm chất ô nhiễm được xem là đúng. Khác nhau sẽ là sai

In [9]:
def loss_function(y_pred, y_test):
    count_diff = 0
    for i in range(len(y_test)):
        aqi_predict = identify_pollutants(y_pred.loc[i], aqi_thresholds, air)
        aqi_test = identify_pollutants(y_test.iloc[i].values, aqi_thresholds, air)
        if group_pollutant(aqi_predict) != group_pollutant(aqi_test):
            count_diff += 1

    return count_diff / len(y_test)

Tính MSE và RMSE giữa tập dự đoán và tập test

In [10]:
def calculate_MSE_RMSE(df, y_test):
    scaler = StandardScaler()
    tmp_df = df.to_numpy()
    tmp_y = y_test.to_numpy()
    tmp_df = scaler.fit_transform(tmp_df.reshape(-1, 1))
    tmp_y = scaler.transform(tmp_y.reshape(-1, 1))

    # Tính MSE và RMSE
    mse = mean_squared_error(tmp_df, tmp_y)
    rmse = np.sqrt(mse)

    print('MSE: ', mse)
    print('RMSE: ', rmse)

#### Chia tập
Chia tập dữ liệu theo tỉ lệ 80/10/10: Mô hình có đủ dữ kiện để học, tinh chỉnh tham số và kiểm tra xem mô hình có bị overfitting/underfitting hay không

In [11]:
# Chia dữ liệu thành tập train (80%), validation (10%) và test (10%)
# Tách tập test (10%)
X_train_valid, X_test, y_train_valid, y_test = train_test_split(features, target, test_size=0.1, random_state=42)

# Tách tập train và valid (1/9 của train_valid)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.1/0.9, random_state=42)

### Mô hình cơ sở: Hồi quy tuyến tính đơn biến

Vì mô hình hồi quy tuyến tính không có tham số điều chỉnh do đó dùng tập dữ liệu train/test đã được chia ở trên để huấn luyện mô hình.

In [12]:
y_pred_lg = []

for col in y_train.columns:
    model = LinearRegression()
    model.fit(X_train, y_train[col])
    y_pred_lg.append(model.predict(X_test))

# Chuyển về dataframe
y_pred_lg = pd.DataFrame(y_pred_lg).T
y_pred_lg.columns = y_train.columns

In [13]:
y_pred_lg

,so2,no2,pm10,pm2_5,o3,co
0,53.117894,45.676705,82.350862,56.901558,30.778086,1429.015557
1,34.525836,33.706021,76.990864,51.969345,-2.825463,1398.166862
2,38.266758,36.520278,73.721277,52.567405,9.193227,1368.054979
3,49.813932,48.180567,90.691796,61.524219,26.961112,1513.357948
4,50.544316,42.239755,90.905761,64.761794,9.355157,1627.756462
...,...,...,...,...,...,...
790,37.268364,33.065534,53.267722,40.668780,11.031866,1173.083918
791,48.504891,41.502122,77.451257,55.096870,44.173248,1286.875518
792,33.814372,30.822428,37.777394,29.719670,14.045136,986.054400
793,34.817757,32.633543,35.839401,26.989596,24.144985,907.845988


##### Tính loss của mô hình

In [14]:
loss = loss_function(y_pred_lg, y_test)
print('Loss: ', loss)

Loss:  0.2842767295597484


<ipython-input-7-239c5ca74880>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i


In [15]:
calculate_MSE_RMSE(y_pred_lg, y_test)

MSE:  0.5256089999402552
RMSE:  0.7249889653920639


#### Mô hình cơ sở: Cây quyết định

Mô hình cây quyết định sử dụng thêm tập dữ liệu valid để tinh chỉnh siêu tham số

In [16]:
validation_loss = []
max_depth = 31

for depth in range(1, max_depth):
    # Huấn luyện
    model = DecisionTreeRegressor(max_depth=depth, random_state=42)
    model.fit(X_train, y_train)
    # Dự đoán trên valid
    y_pred_valid = model.predict(X_valid)
    # Chuyển sang df
    y_pred_valid_df = pd.DataFrame(y_pred_valid, columns=y_train.columns)
    # Tính loss
    loss = loss_function(y_pred_valid_df, y_valid)
    # Lưu kết quả
    validation_loss.append((depth, loss))

best_depth, best_loss = min(validation_loss, key = lambda x: x[1])
print(f"Best max_depth: {best_depth}, Validation Loss: {best_loss}")

<ipython-input-7-239c5ca74880>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i
<ipython-input-7-239c5ca74880>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i
<ipython-input-7-239c5ca74880>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i
<ipython-input-7-239c5ca7

Best max_depth: 6, Validation Loss: 0.22515723270440252


##### Huấn luyện mô hình cuối cùng

In [17]:
final_model = DecisionTreeRegressor(max_depth=best_depth, random_state=42)
final_model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=6, random_state=42)

In [18]:
y_pred_test = final_model.predict(X_test)
y_pred_test_df = pd.DataFrame(y_pred_test, columns=y_train.columns)
y_pred_test_df

,so2,no2,pm10,pm2_5,o3,co
0,48.304573,44.051829,71.544939,44.867530,25.277500,1334.391159
1,40.030438,37.207828,59.003156,44.335750,17.249750,1240.620000
2,43.961887,36.331051,87.261968,64.942049,7.518437,1589.127332
3,41.916136,42.385455,55.207727,40.831818,30.718182,969.041136
4,52.111039,45.556371,142.455858,94.797637,2.870642,2164.191295
...,...,...,...,...,...,...
790,32.750944,32.892296,32.945873,21.775585,28.519056,753.589873
791,60.712923,58.100615,81.891231,64.226385,40.724154,1548.356923
792,32.750944,32.892296,32.945873,21.775585,28.519056,753.589873
793,40.030438,37.207828,59.003156,44.335750,17.249750,1240.620000


In [19]:
# loss
loss = loss_function(y_pred_test_df, y_test)
print('Loss: ', loss)

Loss:  0.22767295597484277


<ipython-input-7-239c5ca74880>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i


In [20]:
calculate_MSE_RMSE(y_pred_test_df, y_test)

MSE:  0.474362030102614
RMSE:  0.6887394500844379


### Mô hình hồi quy đa biến

#### Chia tập
Mô hình hồi quy đa biến không sử dụng rừng ngẫu nhiên làm mô hình cơ sở, việc chia train/valid được thực hiện tự động bởi hàm GridSearchCV dựa trên tập dữ liệu đầu vào là X_train, y_train đã chia ở trên. Grid search sử dụng k-fold cross-validation để tinh chỉnh siêu tham số tự động

### Phần này mô hình chạy tương đối lâu

In [21]:
# Định nghĩa mô hình RandomForestRegressor
rf = RandomForestRegressor()

# Định nghĩa mô hình MultiOutputRegressor với RandomForestRegressor
model = MultiOutputRegressor(rf)

# Tạo một GridSearchCV với các tham số cần tinh chỉnh
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [10, 20, None],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

# Huấn luyện mô hình và tìm tham số tốt nhất
grid_search.fit(X_train, y_train)

# In ra các tham số tối ưu
print("Best parameters found: ", grid_search.best_params_)

# Dự đoán với mô hình đã được tối ưu
predictions = grid_search.predict(X_test)

Best parameters found:  {'estimator__max_depth': 20, 'estimator__min_samples_leaf': 4, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 200}


In [22]:
pred_df = pd.DataFrame(predictions, columns=y_train.columns)
pred_df

,so2,no2,pm10,pm2_5,o3,co
0,71.511263,52.766951,74.755871,60.076883,23.513604,1600.800570
1,34.753562,34.333620,49.892507,36.027918,12.612016,841.541682
2,41.441202,29.332640,97.072816,65.004858,1.314496,1588.477226
3,41.408827,37.705734,50.566299,32.407284,17.170318,1069.066526
4,62.831525,48.985414,156.317847,94.679247,1.509152,1883.064466
...,...,...,...,...,...,...
790,35.878281,33.798403,35.207106,24.944268,6.691121,789.744365
791,54.447827,52.150829,89.616087,74.051980,55.082411,1394.919552
792,37.154756,34.368471,37.881935,26.566631,12.142765,905.540398
793,33.946566,29.772031,50.566672,41.003219,10.432796,1262.039202


In [23]:
loss = loss_function(pred_df, y_test)
print('Loss: ', loss)

Loss:  0.22641509433962265


<ipython-input-7-239c5ca74880>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  value = row[i]  # Giá trị của chất ô nhiễm tại vị trí i


In [27]:
calculate_MSE_RMSE(pred_df, y_test)

MSE:  0.36887582942573455
RMSE:  0.6073514875471488


### Nhận xét về Loss:
Hàm Loss do nhóm thiết kế phần nào phản ánh mong muốn xác định loại ô nhiêm của từng mẫu không khí mà nhóm đã đặt ra. Hàm Loss có tương quan thuận với MSE và RMSE.

### Nhận xét về MSE và RMSE:
- Mô hình hồi quy tuyến tính đơn biến:
  + MSE: 0.5256, RMSE: 0.7250.
  + Đây là mô hình đơn giản nhất, chỉ sử dụng một biến đầu vào để dự đoán. Kết quả cho thấy độ chính xác thấp nhất trong ba mô hình, thể hiện qua MSE và RMSE cao hơn so với các mô hình khác.
- Mô hình cây quyết định:
  + MSE: 0.4744, RMSE: 0.6887.
  + So với hồi quy tuyến tính đơn biến, cây quyết định có MSE và RMSE thấp hơn, cho thấy nó dự đoán tốt hơn. Điều này là nhờ cây quyết định có khả năng bắt các quan hệ phi tuyến tính trong dữ liệu.
- Mô hình hồi quy đa biến:
  + MSE: 0.3689, RMSE: 0.6074.
  + Đây là mô hình có độ chính xác cao nhất, với MSE và RMSE thấp nhất. Lý do chính là mô hình này sử dụng nhiều biến đầu vào, cho phép nắm bắt tốt hơn các yếu tố ảnh hưởng đến biến mục tiêu.

### Kết luận:
- Mô hình hồi quy đa biến hoạt động tốt nhất trong ba mô hình, vì nó sử dụng nhiều thông tin hơn để cải thiện độ chính xác.
- Mô hình cây quyết định đứng thứ hai, đặc biệt hữu ích nếu dữ liệu có mối quan hệ phi tuyến.
- Mô hình hồi quy tuyến tính đơn biến có độ chính xác thấp nhất, do giới hạn trong việc chỉ sử dụng một biến đầu vào.